In [2]:
from importlib.metadata import version

print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

torch version: 2.6.0
tiktoken version: 0.9.0


In [4]:
import tiktoken
from pythainlp.tokenize import word_tokenize

tokenizer = tiktoken.get_encoding("gpt2")

In [5]:
integers = tokenizer.encode("สวัสดีครับคนไทย ทดสอบการ tokenize ด้วย tiktoken")
print(integers)

[19567, 103, 19567, 100, 19567, 109, 19567, 103, 19567, 242, 19567, 113, 19567, 226, 19567, 96, 19567, 109, 19567, 248, 19567, 226, 19567, 247, 31479, 226, 19567, 245, 19567, 95, 220, 19567, 245, 19567, 242, 19567, 103, 19567, 255, 19567, 248, 19567, 223, 19567, 110, 19567, 96, 11241, 1096, 220, 19567, 242, 31479, 231, 19567, 100, 19567, 95, 256, 1134, 30001]


In [7]:
tokenizer.decode(integers)

'สวัสดีครับคนไทย ทดสอบการ tokenize ด้วย tiktoken'

In [16]:
import tiktoken
import torch
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]


def create_dataloader(txt, batch_size=4, max_length=256, stride=128):
    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(dataset, batch_size=batch_size)

    return dataloader

with open("../the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

tokenizer = tiktoken.get_encoding("gpt2")
encoded_text = tokenizer.encode(raw_text)

In [17]:
dataloader = create_dataloader(raw_text, batch_size=4, max_length=2, stride=2)

for batch in dataloader:
    x, y = batch
    break

x

tensor([[19567,   249],
        [19567,    98],
        [19567,   110],
        [19567,   232]])

In [13]:
dataloader = create_dataloader(raw_text, batch_size=4, max_length=8, stride=2)

for batch in dataloader:
    x, y = batch
    break

x

tensor([[19567,   249, 19567,    98, 19567,   110, 19567,   232],
        [19567,    98, 19567,   110, 19567,   232, 19567,   247],
        [19567,   110, 19567,   232, 19567,   247, 19567,   112],
        [19567,   232, 19567,   247, 19567,   112, 19567,   242]])